In [1]:
import sys,tweepy,csv,re
import matplotlib.pyplot as plt
import pylab as plb
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 
from sklearn import naive_bayes
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
from sklearn import cross_validation

pd.set_option('display.notebook_repr_html', True)

class SentimentAnalysis:

    def MainProcess(self):
        
        df = pd.DataFrame()
        df_corpus = pd.DataFrame()
        df_tfidf = pd.DataFrame()
        df_result_pred = pd.DataFrame()
        #################################
        df = pd.read_csv('zidane_resample.csv')
        doc = 'Zidane_Review'
        #################################
        df = df.sample(frac=1).reset_index(drop=True)
        #display (df)
        
        ###########################################  after delete the digit
        #df_tfidf = pd.read_csv('corpus-honda-clean.csv')
        #display (df_tfidf)
        ###########################################
        
        #df_corpus = self.Corpus(df.Tweet)
        #display (df_corpus)
        
        pos_count = 0
        neg_count = 0
        
        for x in df.Polarity_Class:
            if x == 'Positive':
                pos_count += 1
            elif x == 'Negative':
                neg_count += 1
                
        
        iteration = 10
        max_accuracy = 0
        max_con_matrix = 0
        max_report = 0
        max_iter = 0
        kfold_accuracy = 0
        kfold_con_matrix = 0
        kfold_report = 0
        kfold_fold = 0
        kfold_iter = 0
        fkfold_accuracy = 0
        fkfold_con_matrix = 0
        fkfold_report = 0
        fkfold_fold = 0
        fkfold_iter = 0
        store = []
        df_store = pd.DataFrame()
        
        
        # ITERATION
        
        ###############################################################################################    
        print ('______________________________________________________________________________\n') 
        print ('STANDARD CLASSIFICATION :\n')       
        for i in range (iteration):
            
            test_size,accuracy,con_matrix,report = self.NaiveBayes(df.Polarity_Class, df.Tweet)
            if float(accuracy) > float(max_accuracy):
                max_accuracy = accuracy
                max_con_matrix = con_matrix
                max_report = report
                max_iter = str(i+1)
                
            print ('Accuracy on iteration ' +str(i+1)+ ' : ' +str(format(accuracy, '.3f')))
           
        print ('\nITERATION COMPLETE!') 
        print ('\n______________________________________________________________________________')
        print ('\nDocuments name : ' +str(doc)+ '.csv') 
        print ('\nAccuracy : '+ str(format(max_accuracy, '.3f')))
        print ('Iteration number : ' +str(max_iter))
        print ('\nConfusion Matrix : \n\n' +str(max_con_matrix))
        print ('\nClassification Report : \n\n' +str(max_report))
        print ('______________________________________________________________________________\n')
        
        print ('\nCLASSIFICATION ON CROSS VALIDATION :\n')       
        for i in range (iteration):
            
            kmax_accuracy,kmax_con_matrix,kmax_report,kno_fold = self.KFold(df.Polarity_Class, df.Tweet)
            if float(kmax_accuracy) > float(kfold_accuracy):
                kfold_accuracy = kmax_accuracy
                kfold_con_matrix = kmax_con_matrix
                kfold_report = kmax_report
                kfold_fold = kno_fold
                kfold_iter = str(i+1)
            print ('Accuracy on iteration ' +str(i+1)+ ' with maximum at fold ' +str(kno_fold)+ ' : ' +str(format(kmax_accuracy, '.3f')))
                
            df = df.sample(frac=1).reset_index(drop=True)
            
        print ('\nITERATION COMPLETE!') 
        print ('\n______________________________________________________________________________')
        print ('\nDocuments name : ' +str(doc)+ '.csv')
        print ('Highest accuracy on KFold Validation is on fold '+str(kfold_fold)+' in iteration ' +str(kfold_iter)+ ' :')
        print ('Accuracy : '+str(format(kfold_accuracy, '.3f')))
        print ('\nConfusion Matrix : \n\n' +str(kfold_con_matrix))
        print ('\nClassification Report : \n\n' +str(kfold_report))
        print ('______________________________________________________________________________')
        
        
        ###############################################################################################

                                                                                                          
        '''
        # NO ITERATION
        ###############################################################################################
        test_size,accuracy,con_matrix,report = self.NaiveBayes(df.Polarity_Class, df.Tweet)
        
        print ('\nDocuments name : ' +str(doc)+ '.csv')
        print ('Testing size : ' +str(test_size*100)+'%')
        print ('Total positive tweets : ' +str(pos_count))
        print ('Total negative tweets : ' +str(neg_count))
        print ('\nAccuracy : '+str(format(accuracy, '.3f')))   
        print ('\nConfusion Matrix : \n' +str(con_matrix))
        print ('\nClassification Report : \n' +str(report))
      
        
        max_accuracy,max_con_matrix,max_report,no_fold = self.KFold(df.Polarity_Class, df.Tweet)
        
        print ('\nDocuments name : ' +str(doc)+ '.csv')
        print ('\nHighest accuracy on KFold Validation is on fold '+str(no_fold)+' :')
        print ('Accuracy : '+str(format(max_accuracy, '.3f')))
        print ('\nConfusion Matrix : \n' +str(max_con_matrix))
        print ('\nClassification Report : \n' +str(max_report))
                
        ###############################################################################################
        '''
        
        
    def Corpus(self, Tweet):
        
        doc = []
        vectorizer = TfidfVectorizer(use_idf=True, strip_accents='ascii')
        
        for x in Tweet:
            doc.append((str(x)).lower())         
        
        stop_words = set(stopwords.words('english'))
        tokens = word_tokenize(str(doc))
        
        fw = [w for w in tokens if not w in stop_words]
        fw = []
        #filtered_words = [x for x in fw if not any(c.isdigit() for c in x)]
        filtered_words = [item for item in fw if item.isalpha()]
        filtered_words = []
        
        ps = PorterStemmer()
        for t in tokens:
            if t not in stop_words:
                w = ps.stem(t)
                filtered_words.append(w)  
                
        temp = vectorizer.fit_transform(filtered_words) 
        
        vocab = vectorizer.get_feature_names()
        temp = temp.toarray()
        dist = np.sum(temp, axis=0)
        df = pd.DataFrame()
        list1 = []
        list2 = []
        
        for w,c in zip(vocab, dist):
            list1.append(w)
            list2.append(c)
            
        
      
        df['Vocabulary'] = list1
        #df['Word_count'] = list2
        df['TF-IDF'] = list2
        
        df.to_csv('corpus-honda.csv', sep=',', index=False, encoding='utf-8')
        
        return df
    
    def KFold(self, Class, Tweet):
        
        stopset = set(stopwords.words('english'))
        vectorizer = TfidfVectorizer(use_idf=True, lowercase=True, strip_accents='ascii', stop_words=stopset)
        
        y = Class
        X = vectorizer.fit_transform(Tweet)
        
        kf = KFold(n_splits=10) 
        kf.get_n_splits(X) 
        
        clf = naive_bayes.MultinomialNB()
        
        count = 1
        count = int(count)
        
        max_accuracy = 0
        max_con_matrix = 0
        max_report = 0
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            
            #print ('\nAccuracy on K-fold Validation ' +str(count)+ ' : ')
            accuracy = accuracy_score(y_test, y_pred)
            con_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
            report = classification_report(y_test, y_pred)
            
            #print ('Accuracy : '+str(format(accuracy, '.3f')))
            #print ('\nConfusion Matrix : \n' +str(con_matrix))
            #print ('\nClassification Report : \n' +str(report))
            
            if float(accuracy) > float(max_accuracy):
                max_accuracy = accuracy
                max_con_matrix = con_matrix
                max_report = report
                no_fold = count
            count +=1
            
        return max_accuracy, max_con_matrix, max_report, no_fold
            

    def NaiveBayes(self, Class, Tweet):
        
        stopset = set(stopwords.words('english'))
        vectorizer = TfidfVectorizer(use_idf=True, lowercase=True, strip_accents='ascii', stop_words=stopset)
      
        y = Class
        X = vectorizer.fit_transform(Tweet)
        
        #test_size = float(raw_input("Enter the percentage of testing set: "))
        test_size = 0.3
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=test_size)
        clf = naive_bayes.MultinomialNB()
        clf.fit(X_train, y_train)
        
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        con_matrix = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
        report = classification_report(y_test, y_pred)
        
        return test_size,accuracy,con_matrix,report

        
if __name__== "__main__":
    sa = SentimentAnalysis()
    sa.MainProcess() 

C:\Users\User\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


______________________________________________________________________________

STANDARD CLASSIFICATION :

Accuracy on iteration 1 : 0.834
Accuracy on iteration 2 : 0.813
Accuracy on iteration 3 : 0.832
Accuracy on iteration 4 : 0.817
Accuracy on iteration 5 : 0.817
Accuracy on iteration 6 : 0.840
Accuracy on iteration 7 : 0.817
Accuracy on iteration 8 : 0.841
Accuracy on iteration 9 : 0.825
Accuracy on iteration 10 : 0.832

ITERATION COMPLETE!

______________________________________________________________________________

Documents name : Car_Review.csv

Accuracy : 0.841
Iteration number : 8

Confusion Matrix : 

Predicted  Negative  Positive  All
Actual                            
Negative        282        48  330
Positive         59       284  343
All             341       332  673

Classification Report : 

             precision    recall  f1-score   support

   Negative       0.83      0.85      0.84       330
   Positive       0.86      0.83      0.84       343

avg / total   